## 13 · Reverse Diffusion Sampler  
A simple Euler-style loop that starts from Gaussian noise and iteratively subtracts the model’s predicted noise.  
Not production-grade sampling, but perfect for illustrating inference after training.

In [ ]:
# 13. Reverse Diffusion Sampling

def sample_image(model, steps=50, device="cpu"):
    """Generate an image by iteratively de-noising random noise."""
    model.eval()
    with torch.no_grad():
        img = torch.randn(1, 3, 224, 224, device=device)
        for step in reversed(range(steps)):
            t = torch.tensor([step], device=device)
            pred_noise = model(img, t)
            img = img - pred_noise * 0.1                      # simple Euler update
        # Rescale back to [0,1]
        img = torch.clamp((img * 0.5 + 0.5), 0.0, 1.0)
        return img.squeeze(0).cpu().permute(1,2,0).numpy()

### 14 · Generate & Display Samples from **best checkpoint**  
Load the model weights from `best_ckpt`, move to GPU if available, generate three images, and show them side-by-side.  
Remember: with a tiny CNN and only 10 epochs, these samples look noise-like. If you replace the backbone or train longer you'd expect to see better quality.


In [ ]:
# 14. Generate & Display Samples

# Load model from Ray Train checkpoint
from ray.train import Checkpoint

assert best_ckpt is not None, "Checkpoint is missing. Did training run and complete?"

with best_ckpt.as_directory() as ckpt_dir:
    model = PixelDiffusion()
    model.load_state_dict(torch.load(os.path.join(ckpt_dir, "model.pt"), map_location="cpu"))

model = model.to("cuda" if torch.cuda.is_available() else "cpu")

# Generate three images
samples = [sample_image(model, steps=50, device=model.device) for _ in range(3)]

fig, axs = plt.subplots(1, 3, figsize=(9, 3))
for ax, img in zip(axs, samples):
    ax.imshow(img)
    ax.axis("off")
plt.suptitle("Food‑101 Diffusion Samples (unconditional)")
plt.tight_layout()
plt.show()

### 15 · Cleanup Shared Storage  
Reclaim cluster disk space by deleting the entire tutorial output directory.  
Run this only when you’re **sure** you don’t need the checkpoints or metrics anymore.

In [ ]:
# 15. Cleanup -- delete checkpoints and metrics from model training

TARGET_PATH = "/mnt/cluster_storage/generative_cv"

if os.path.exists(TARGET_PATH):
    shutil.rmtree(TARGET_PATH)
    print(f"✅ Deleted everything under {TARGET_PATH}")
else:
    print(f"⚠️ Path does not exist: {TARGET_PATH}")